In [27]:
# 모듈 전체를 먼저 import
from bigquery import BigQuery

# 모듈을 reload
import importlib
importlib.reload(bigquery)

# reload 이후 다시 클래스 import (권장)
from bigquery import BigQuery

bq = BigQuery(projectCode='sleeper')
df = bq.get_data('tb_sleeper_psi')

Query Start... 최대 60초 대기합니다.


Processing...: 100%|██████████| 00:01


총 예상 행 수: 209912


In [30]:
import numpy as np

df["_sourceMedium"] = df["psi_source"].astype(str) + " / " + df["psi_medium"].astype(str)
# df["_isUserNew_y"] = np.where(df["first_visit"] == 1, df["user_pseudo_id"], np.nan)
# df["_isUserNew_n"] = np.where(df["first_visit"] == 0, df["user_pseudo_id"], np.nan)
df["_isUserNew_y"] = (df["first_visit"] == 1).astype(int)
df["_isUserNew_n"] = (df["first_visit"] == 0).astype(int)

df["_engagement_time_sec_sum"] = df["engagement_time_msec_sum"] / 1000

# df["_find_nearby_showroom_sessionCnt"] = np.where(df["find_nearby_showroom"] > 0, df["pseudo_session_id"], np.nan)
# df["_product_option_price_sessionCnt"] = np.where(df["product_option_price"] > 0, df["pseudo_session_id"], np.nan)
# df["_view_item_sessionCnt"] = np.where(df["view_item"] > 0, df["pseudo_session_id"], np.nan)
# df["_add_to_cart_sessionCnt"] = np.where(df["add_to_cart"] > 0, df["pseudo_session_id"], np.nan)
# df["_purchase_sessionCnt"] = np.where(df["purchase"] > 0, df["pseudo_session_id"], np.nan)
# df["_showroom_10s_sessionCnt"] = np.where(df["showroom_10s"] > 0, df["pseudo_session_id"], np.nan)
# df["_product_page_scroll_50_sessionCnt"] = np.where(df["product_page_scroll_50"] > 0, df["pseudo_session_id"], np.nan)

df["_find_nearby_showroom_sessionCnt"] = (df["find_nearby_showroom"] > 0).astype(int)
df["_product_option_price_sessionCnt"] = (df["product_option_price"] > 0).astype(int)
df["_view_item_sessionCnt"] = (df["view_item"] > 0).astype(int)
df["_add_to_cart_sessionCnt"] = (df["add_to_cart"] > 0).astype(int)
df["_purchase_sessionCnt"] = (df["purchase"] > 0).astype(int)
df["_showroom_10s_sessionCnt"] = (df["showroom_10s"] > 0).astype(int)
df["_product_page_scroll_50_sessionCnt"] = (df["product_page_scroll_50"] > 0).astype(int)



def map_isPaid_4(row):
    source_medium = row["_sourceMedium"]
    psi_source = row["psi_source"]

    if source_medium in ['google / organic', 'naver / organic']:
        return 'ETC'
    elif psi_source in [
        'google', 'naver', 'meta', 'meta_adv', 'mobon', 'mobion',
        'naver_gfa', 'DV360', 'dv360', 'fb', 'sns', 'IGShopping',
        'criteo'
    ] or source_medium in ['youtube / demand_gen', 'kakako / crm']:
        return 'Paid'
    elif psi_source in [
        'litt.ly', 'instagram', 'l.instagram.com', 'instagram.com',
        'blog.naver.com', 'm.blog.naver.com', 'smartstore.naver.com', 'm.brand.naver.com'
    ] or source_medium == 'kakao / channel_message':
        return 'Owned'
    elif psi_source in ['youtube', 'youtube.com', 'm.youtube.com'] or source_medium in [
        'm.facebook.com / referral', 'l.facebook.com / referral', 'facebook.com / referral'
    ]:
        return 'Earned'
    else:
        return 'ETC'

df["isPaid_4"] = df.apply(map_isPaid_4, axis=1)


In [32]:
df.columns

Index(['pseudo_session_id', 'user_pseudo_id', 'ga_session_id', 'first_visit',
       'sign_up', 'view_item', 'add_to_cart', 'wishlist', 'view_cart',
       'begin_checkout', 'purchase', 'page_view', 'scroll', 'session_start',
       'user_engagement', 'click_category_depth_1', 'click_category_depth_2',
       'click_category_depth_3', 'scroll_50', 'product_page_scroll_50',
       'sign_up_start', 'click_banner_top', 'select_promotion',
       'click_promotion_cta', 'view_item_list', 'select_item', 'showroom_10s',
       'showroom_leads', 'find_nearby_showroom', 'product_option_price',
       'purchase_button_click', 'min_num', 'max_num', 'event_ts', 'event_date',
       'geo__sub_continent', 'geo__country', 'geo__city', 'device__category',
       'device__operating_system', 'engagement_time_msec_sum', 'psi_source',
       'psi_medium', 'psi_campaign', 'psi_content', '_sourceMedium',
       '_isUserNew_y', '_isUserNew_n', '_engagement_time_sec_sum',
       '_find_nearby_showroom_session